In [ ]:
import sys
input = lambda: sys.stdin.readline().rstrip()

A

In [7]:
import sys
input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n = int(input())
    li = list(map(int, input().split()))

    init = li[0]
    stack = []
    for i in range(1, n):
        while stack and stack[-1] < li[i]:
            stack.pop()

        if not stack and init < li[i]:
            continue
        else:
            stack.append(li[i])

    if stack:
        print('NO')
    else:
        print('YES')

4
2 1 3 4
YES


C

In [15]:
import sys
input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n = int(input())
    n *= 2
    li = list(map(int, input().split()))

    nli = [(v, i) for i, v in enumerate(li)]
    nli.sort(key=lambda x: -x[0])

    cur_left = n
    groups = []
    for value, index in nli:
        if index > cur_left:
            continue
        groups.append(cur_left - index)
        cur_left = index

    n_half = n // 2
    dp = [False] * (n_half + 1)
    dp[0] = True
    for v in groups:
        if v > n_half:
            continue
        for i in range(n_half, v - 1, -1):
            if dp[i - v]:
                dp[i] = True

    if dp[n_half]:
        print('YES')
    else:
        print('NO')

4
8 7 6 5 4 3 2 1


B

In [39]:
import sys
# input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n, k = map(int, input().split())
    li = list(map(int, input().split()))

    ip = True
    l, h = li[0], li[0] + k - 1 + k
    for i in range(1, n):
        nl = li[i]
        nh = li[i] + k - 1 + k
        if i == n - 1:
            nh = li[i] + k
        
        nl = max(nl, l + 1 - k)
        nh = min(nh, h - 1 + k)
        
        if nh - nl < k:
            ip = False
            break
            
        l, h = nl, nh

    if ip:
        print('YES')
    else:
        print('NO')

1
5 6
1 2 1 2 1
1 12 2 13
2 13 1 12
1 12 2 13
2 13 1 12
YES


D

In [ ]:
v^k는 v^0 + ... + v^(k - 1)보다 더 큼

그러므로 필요한 k가 서로 중복되면 안됨

In [50]:
4 ** 5 + 4 ** 7 + 4 + 1

17412

In [59]:
import sys
input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n, k = map(int, input().split())
    li = list(map(int, input().split()))

    st = set()
    ip = True
    for v in li:
        count = 0
        while v:
            temp = v % k
            if temp == 1:
                if count in st:
                    ip = False
                    break
                st.add(count)
                v -= temp
            elif temp:
                ip = False
                break
            v //= k
            count += 1
        if not ip:
            break

    if ip:
        print('YES')
    else:
        print('NO')

3 9
59049 0 810
NO


In [64]:
v = 59049
k = 9
st = set()

count = 0
while v:
    temp = v % 4
    if temp == 1:
        if count in st:
            ip = False
            break
        st.add(count)
        v -= temp
    elif temp:
        ip = False
        break
    v //= 4
    count += 1
    
st

1
2


{0}

E

In [66]:
import sys
input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n = int(input())
    li = list(map(int, input().split()))

    diffs = []
    for i in range(n):
        if li[i] != i + 1:
            diffs.append(i)

    if not diffs:
        print(0)
    else:
        r = 1
        for i in range(len(diffs) - 1):
            if diffs[i] + 1 != diffs[i + 1]:
                r = 2
                break
        print(r)

7
3 2 4 5 1 6 7
2


F

In [ ]:
import sys
input = lambda: sys.stdin.readline().rstrip()

## PYRIVAL BOOTSTRAP
# https://github.com/cheran-senthil/PyRival/blob/master/pyrival/misc/bootstrap.py
# This decorator allows for recursion without actually doing recursion
## @bootstrap, yield when getting and returning value in recursive functions, end of functions
from types import GeneratorType

def bootstrap(f, stack=[]):
    def wrappedfunc(*args, **kwargs):
        if stack:
            return f(*args, **kwargs)
        else:
            to = f(*args, **kwargs)
            while True:
                if type(to) is GeneratorType:
                    stack.append(to)
                    to = next(to)
                else:
                    stack.pop()
                    if not stack:
                        break
                    to = stack[-1].send(to)
            return to
    return wrappedfunc

@bootstrap
def dfs(value):
    global min_count
    global min_count_values
    
    vt[value] = True
    
    cumul = 0
    max_c = 0
    for v in li[value]:
        if vt[v]:
            continue
        c = yield dfs(v)
        cumul += c
        max_c = max(max_c, c)
    max_c = max(max_c, n - 1 - cumul)
    
    if max_c < min_count:
        min_count = max_c
        min_count_values = [value]
    elif max_c == min_count:
        min_count_values.append(value)
        
    yield cumul + 1

@bootstrap    
def dfs_to_find_leaf(value):
    global to_delete
    
    vt[value] = True
    
    count = 0
    for v in li[value]:
        if vt[v]:
            continue
        c = yield dfs_to_find_leaf(v)
        if c == 1:
            if not to_delete:
                to_delete = [value, v]
        count += c
    yield count + 1

for _ in range(int(input())):
    n = int(input())
    li = [[] for _ in range(n + 1)]
    for _ in range(n - 1):
        x, y = map(int, input().split())
        li[x].append(y)
        li[y].append(x)

    min_count = float('inf')
    min_count_values = []
    vt = [False] * (n + 1)
    dfs(1)

    if len(min_count_values) == 1:
        print(1, li[1][0])
        print(1, li[1][0])
    else:   
        vt = [False] * (n + 1)
        vt[min_count_values[1]] = True
        to_delete = []
        dfs_to_find_leaf(min_count_values[0])
        print(*to_delete)
        print(to_delete[1], min_count_values[1])

G

In [ ]:
x의 값은 커질 수만 있을뿐 작아질 수는 없다.
a[i]의 값은 작아질 수만 있을뿐 커질 수는 없다.



In [ ]:
바꿀 수 있으면 무조건 바꾸고 정렬된 상태면 중단?

In [49]:
import sys
input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n, x = map(int, input().split())
    li = list(map(int, input().split()))

    if li == sorted(li):
        print(0)
    else:
        c = 0
        ip = False
        for i in range(n):
            if li[i] > x:
                li[i], x = x, li[i]
                c += 1
            if li == sorted(li):
                ip = True
                break

        if ip:
            print(c)
        else:
            print(-1)

5 18
81 324 218 413 324
3


In [43]:
target

[2, 3, 4, 5]